# get int addition response from gemini

Install necessary packages

In [1]:
# !pip install -q -U google-generativeai
# !pip install python-dotenv

In [34]:
import os
from dotenv import load_dotenv
import random
import time
import json
import google.generativeai as genai

### load model

In [5]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
gemini = genai.GenerativeModel('gemini-pro')

### load data and create initial prompt

In [10]:
with open('../../data/int_addition.json','r') as file:
    data = json.load(file)

# create batches, batch sizes are heuristics and I tried out several
# batch sizes, although input token limit is 30,720 tokens, output token limit is 2048
# which restricts the batch size

batch_prompts = []

index = 0
while index < len(data):
    if index < 1000:
        num_prompts = 75
    elif index < 2000:
        num_prompts = 60
    elif index < 3000:
        num_prompts = 40
    elif index < 4000:
        num_prompts = 25
    elif index < 6000:
        num_prompts = 15
    elif index < 10000:
        num_prompts = 12
    elif index< 15000:
        num_prompts = 8
    else:
        num_prompts = 5
    
    batch_prompt = data[index:min(index+num_prompts, len(data))]
    batch_prompts.append(batch_prompt)
    index += num_prompts

print("Number of batches:", len(batch_prompts))
print("Number of prompts in each batch:", [len(batch) for batch in batch_prompts])


Number of batches: 2986
Number of prompts in each batch: [75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12, 12, 12, 12, 12

In [30]:
# give a prompt once and then generate the response,
# and then use this prompt+response directly instead of doing everytime

promt_saying_you_are_a_math_assistant = '''You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 'What is 2 + 3?',you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?'''
Initial_messages = [{'role':'user', 'parts': [promt_saying_you_are_a_math_assistant] }]

# get response from the model and then append it to the prompt
Initial_response = gemini.generate_content(Initial_messages)                                   
Initial_messages.append(Initial_response.candidates[0].content)

In [32]:
print(Initial_messages)

[{'role': 'user', 'parts': ["You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 'What is 2 + 3?',you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?"]}, parts {
  text: "Yes, I understand. I will provide the answer to each addition question in the correct format with each answer on a separate line.\n\nFor example, if you ask \'What is 2 + 3?\', I will answer \'2 + 3 = 5\'.\n\nIs there anything else I can help you with today?"
}
role: "model"
]


In [35]:
import pickle

# save the initial messages to a file, in case we need to restart the conversation
with open('initial_messages_int_add.pkl', 'wb') as file:
    pickle.dump(Initial_messages, file)

### conduct experiment

In [36]:
# path to save responses
response_file = 'gemini_response/int_addition_response.txt'

# request per minute limit for free api
rpm = 60

In [86]:
for i, batch_prompt in enumerate(batch_prompts):
    if i < 1397:
        continue
    # print status
    print(f"Sending batch {i+1} of {len(batch_prompts)}")

    num_prompts = len(batch_prompt)

    # create prompt for the batch
    curr_prompt = "".join([f"what is {a} + {b} ? \n " for (a,b,c) in batch_prompt])
    
    # create a complete prompt to send to the model
    complete_promt = Initial_messages.copy()
    complete_promt.append({'role':'user', 'parts': [curr_prompt]})

    # get response from the model, candidate_count is 1 because we only need one response,
    # temperature is 0 because we need deterministic responses
    response = gemini.generate_content( complete_promt,
                                        generation_config= genai.GenerationConfig(
                                        candidate_count = 1,
                                        temperature = 0.0,
                                        )
                                )
    
    try:
        # extract text from the response
        model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
    except IndexError:
        print("Error in response")
        model_responses = ["0"]*num_prompts

    # save responses to file
    with open(response_file, 'a') as file:
        for model_response in model_responses:
            file.write(model_response + '\n')

    
    #print the prompt and response separated by space
    print('# of prompts: ',len(batch_prompt),'# of responses: ', len(model_responses))


    for prompt, response in zip(batch_prompt, model_responses):
        print(prompt, response, sep=" ->")
    
    
    print("Received responses for batch", i+1, "of", len(batch_prompts))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    time.sleep(60//rpm) #to avoid rate limit errors

print("Responses saved to", response_file)

Sending batch 1398 of 2986
Error in response
# of prompts:  5 # of responses:  5
[58478379273572828678433613394254222296670, 20218640104066167370327495489173272300418, 78697019377638996048761108883427494597088] ->0
[84513731052749498356346572245065495307667, 17682503440973962044954343990025694543561, 102196234493723460401300916235091189851228] ->0
[37125171150177701088757782033732889334988, 99742136924011392145730637345759677869338, 136867308074189093234488419379492567204326] ->0
[51510396797708338687404770524347382264643, 97058757378773986324304105243447583653782, 148569154176482325011708875767794965918425] ->0
[98125616871093139716357562184170925757233, 38641790275675896835333911801686051968284, 136767407146769036551691473985856977725517] ->0
Received responses for batch 1398 of 2986
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Sending batch 1399 of 2986
# of prompts:  5 # of responses:  9
[23385854356988614553490601498736453473906, 64990062404532598177709318